In [1]:
using CSV
using DataFrames
using Serialization
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20210214.csv"));

In [4]:
bad_ids = begin
    id_counts = combine(groupby(rawdata, :hospital_pk), :hospital_name => (x -> length(unique(x))) => :n_names)
    filter!(x -> x.n_names > 1, id_counts)
    unique(id_counts.hospital_pk)
end
filter!(row -> !(row.hospital_pk in bad_ids), rawdata);

In [5]:
findmissing(x) = (ismissing(x) || x == -999999) ? missing : x;

In [6]:
data_weekly = select(rawdata,
    :hospital_name => :hospital,
    :hospital_pk => :hospital_id,
    :collection_week => :date,
    
    :all_adult_hospital_inpatient_beds_7_day_avg => ByRow(findmissing) => :total_beds,
    :all_adult_hospital_inpatient_bed_occupied_7_day_avg => ByRow(findmissing) => :total_occupancy,
    
    :total_staffed_adult_icu_beds_7_day_avg => ByRow(findmissing) => :icu_beds,
    :staffed_adult_icu_bed_occupancy_7_day_avg => ByRow(findmissing) => :icu_occupancy,
);
sort!(data_weekly, [:hospital, :hospital_id, :date]);

In [7]:
function latest_val(xs)
    xs = filter(x -> !ismissing(x), xs)
    z = length(xs) == 0 ? missing : xs[end]
    z = coalesce(z, 0)
    return z
end;

In [8]:
data_latest = combine(groupby(data_weekly, [:hospital, :hospital_id]), [
    :total_beds => latest_val => :total_beds,
    :total_occupancy => latest_val => :total_occupancy,
    :icu_beds => latest_val => :icu_beds,
    :icu_occupancy => latest_val => :icu_occupancy,
]);

In [9]:
compute_load(a,b) = (a==0) ? 0.0 : (b==0) ? 1.0 : a/b;

In [10]:
insertcols!(data_latest, 5, :total_load => compute_load.(data_latest.total_occupancy, data_latest.total_beds));
insertcols!(data_latest, 8, :icu_load => compute_load.(data_latest.icu_occupancy, data_latest.icu_beds));

In [11]:
metadata = DataFrame(CSV.File("../data/hhs_hospital_meta.csv"));

In [12]:
hospital_locations_data = DataFrame(CSV.File("../data/hhs_hospital_locations.csv"))
select!(hospital_locations_data, :hospital_id, :lat, :long);

In [13]:
data_combined_all = leftjoin(data_latest, metadata, on=:hospital_id, makeunique=true);
data_combined_all = leftjoin(data_combined_all, hospital_locations_data, on=:hospital_id);

In [14]:
data_combined_all.lat = coalesce.(data_combined_all.lat, 0.0);
data_combined_all.long = coalesce.(data_combined_all.long, 0.0);

In [15]:
data_combined = select(data_combined_all,
    :hospitalname => :hospital,
    :hospital => :hhsname,
    :hospital_id,
    :lat,
    :long,
    :total_beds,
    :total_occupancy,
    :total_load,
    :icu_beds,
    :icu_occupancy,
    :icu_load,
)

,hospital,hhsname,hospital_id,lat,long,total_beds,total_occupancy,total_load,icu_beds,icu_occupancy,icu_load
,String?,String,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Abbeville Area Medical Center,ABBEVILLE AREA MEDICAL CENTER,421301,34.1579,-82.3815,18.0,10.9,0.605556,0.0,0.0,0.0
2,Abbeville General Hospital,ABBEVILLE GENERAL HOSPITAL,190034,29.9737,-92.1077,34.3,18.9,0.55102,6.3,5.0,0.793651
3,Abbott Northwestern Hospital,ABBOTT NORTHWESTERN HOSPITAL,240057,44.9526,-93.262,419.1,383.0,0.913863,39.4,39.3,0.997462
4,Abilene Regional Medical Center,ABILENE REGIONAL MEDICAL CENTER,450558,32.3747,-99.744,103.6,98.6,0.951737,16.7,16.7,1.0
5,Abington Memorial Hospital,ABINGTON MEMORIAL HOSPITAL,390231,40.1191,-75.1199,525.0,432.7,0.82419,60.1,49.0,0.815308
6,Abraham Lincoln Memorial Hospital,ABRAHAM LINCOLN MEMORIAL HOSPITAL,141322,40.1535,-89.39,25.0,16.7,0.668,0.0,0.0,0.0
7,Abrazo Arrowhead Campus,ABRAZO ARROWHEAD CAMPUS,030094,33.6544,-112.202,218.4,218.4,1.0,42.1,41.3,0.980998
8,Abrazo Central Campus,ABRAZO CENTRAL CAMPUS,030030,33.5256,-112.102,130.0,130.0,1.0,31.7,31.4,0.990536
9,Abrazo Scottsdale Campus,ABRAZO SCOTTSDALE CAMPUS,030083,33.6393,-111.998,73.1,73.1,1.0,13.9,10.4,0.748201


In [16]:
data_combined |> CSV.write("../data/hhs_current_load.csv");

In [17]:
data_combined_list = collect([NamedTuple(h) for h in eachrow(data_combined)]);

In [18]:
serialize("../data/hhs_current_load.jlser", data_combined_list)